# Projet  **<span style="color: #CC146C">Quiz generator </span>** 💡 - Python pour le data-scientist

#### Auteurs : Adrien Servière, Mélissa Tamine.

L'objectif de ce notebook est de présenter le projet que nous avons effectué dans le cadre de l'unité d'enseignement **Python pour le data-scientist** dispensée à l'ENSAE. Ce projet a été élaboré de manière libre et comporte, comme attendu, un **jeu de données** récupéré et traité, une partie **visualisation** et une partie **modélisation**. 

# Problématique

Notre projet s'articule autour de la problématique suivante : **Comment créer un système capable de générer un quiz (plusieurs paires de question/réponse) sur un thème précis ?**

Dans la mesure où l'objectif principal d'un quiz est d’évaluer les connaissances d’un participant, il nous a semblé qu'un tel système pourrait s'avérer très utile à des enseignants afin de tester de manière ludique les acquis de leurs élèves par exemple. 

C'est pourquoi nous avons modélisé la structure suivante afin que le système créé puisse répondre au problème :

![framework](./data/images/framework.png "Structure du système implémenté")

La structure est divisée en deux parties distinctes : 
1. Une partie **traitement des données** qui a principalement consisté à extraire et indexer dans ElasticSearch la base de données Wikipédia sur laquelle le modèle se fonde.
2. Une partie **modélisation** fondée sur la mise en place d'une *pipeline* formée de plusieurs outils de traitement du langage et modèles de langages.

# Installations et recommandations préalables

Avant d'exécuter veuillez procéder aux installations de modules nécessaires au bon fonctionnement du code en exécutant la cellule ci-dessous. 

In [55]:
!pip install elasticsearch
!pip install wordcloud
!pip install nltk
!pip install spacy
!python3 -m spacy download en
!pip install pywaffle
!pip install stanza
!pip install pytorch_lightning
!pip install sentencepiece
!pip install transformers
!pip install strsimpy

De même, nous vous demanderons d'exécuter les cellules de ce notebook au sein d'un **espace de travail muni d'un service ElasticSearch préalablement exécuté** afin que la partie indexation puisse fonctionner. Nous vous conseillons d'utiliser **SSP Cloud** car la technologie ElasticSearch y est disponible.

# Importation des modules utiles

In [4]:
import os
import sys
import seaborn
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
from src.scripts.wikipedia_indexing import *
from src.data.constants import *
from src.data.visualisation import *
import stanza
from src.scripts.quiz_generator import *

# Récupération et traitement des données 🗂️

## Extraction des données textuelles provenant de l'encyclopédie Wikipédia

La première étape de notre projet a été la **récupération** et le **traitement des données**. Dans notre cas, nous avons fait le choix de récupérer des données textuelles provenant d'articles de l'encyclopédie en ligne **Wikipédia**. 

Pour des raisons de volume, nous avons extrait les données brutes de la version en anglais simple de Wikipédia (en anglais : *Simple English Wikipedia*) plutôt que les versions en anglais ou en français bien trop volumineuses. Il s'agit d'une encyclopédie spécialement fondée pour « des étudiants, des enfants ou des adultes ayant des difficultés de compréhension et pour ceux qui souhaiteraient apprendre l'anglais ». En novembre 2021, date à laquelle nous avons extrait les données brutes, le site contenait plus de **200 000 pages** différentes.

Afin d'extraire ces données brutes et de les convertir en données textuelles pouvant être exploitées, nous avons utilisé l'outil ```Wikiextractor``` (https://attardi.github.io/wikiextractor/) de la manière suivante dans un terminal :
```
>>> wget "http://download.wikimedia.org/simplewiki/latest/simplewiki-latest-pages-articles.xml.bz2"

>>> python -m wikiextractor.WikiExtractor -o "./quiz-generator/data/wikipedia/" --json --processes 12 "quiz-generator/data/wikipedia/simplewiki-latest-pages-articles.xml.bz2"

>>> rm "./quiz-generator/data/wikipedia/simplewiki-latest-pages-articles.xml.bz2"
```

Ces commandes nous ont permis d'obtenir les plus de 200 000 pages de données textuelles sous la forme de plusieurs fichiers .txt formatés comme des données au format JSON que vous trouverez dans le dossier *data* du projet.

## Indexation des données textuelles dans ElasticSearch

Une fois les données récupérées, il a ensuite fallu les traiter. Le traitement a principalement consisté à indexer ces données textuelles dans *ElasticSearch*. 

ElasticSearch c’est un logiciel qui fournit un moteur de recherche installé sur un serveur (dans notre cas le serveur SSP Cloud) qu’il est possible de requêter depuis un client (ce Notebook en l'occurence). C’est un moteur de recherche très performant, puissant et flexible sur données textuelles. L'objectif de l'utilisation d'un tel outil est de trouver, dans un corpus de grande dimension, un certain texte. **Dans notre cas, il s'agit de trouver les textes les plus pertinents sur un thème donné parmi l'ensemble des données textuelles comprises dans les 200 000 pages de données disponibles.**

Nous utilisons la librairie python ```elasticsearch``` pour dialoguer avec notre moteur de recherche Elastic. La ligne de code ci-dessous permet d'établir la connexion avec le cluster Elastic que vous avez dû lancer dans votre session SSP Cloud lors de la phase de recommandation. 

In [5]:
es = set_es_client()

Maintenant que la connexion est établie, nous pouvons passer à l'étape **d'indexation**. Cette étape consiste à envoyer les documents parmi lesquels nous souhaitons chercher des echos pertinents dans notre elastic. Un index est donc une collection de document. Dans notre cas, les documents sont les paragraphes qui composent les articles du *Simple English Wikipedia*. 

**Remarque :** L'exécution de la ligne suivante qui permet l'indexation est relativement longue, veuillez compter environ 7 minutes.

In [40]:
run_indexing(client=es, args=fill_default_args())

## Exécution d'une première requête test

Maintenant que l'étape d'indexation est finalisée, il est désormais possible de lancer notre première **requête** c'est à dire de chercher les documents les plus pertinents à propos d'un thème (un certain mot). 

Pour cela, nous utilisons l'algorithme d'extraction d'information BM25 que nous avons implémenté et qui utilise simplement la méthode interne de pondération des mots utilisée par ElasticSearch. La pertinence d’un mot pour notre recherche est construite sur une variante de la **TF-IDF**, considérant qu’un terme est pertinent s’il est souvent présent dans le document (Term Frequency) alors qu’il est peu fréquent dans les autres document (inverse document frequency).

In [6]:
bm25 = BM25Retriever(client=es)

Nous décidons par exemple de chercher les documents c'est à dire les paragraphes inclus dans le *Simple English Wikipedia* les plus pertinents traitant du philosophe **Emmanuel Kant**.

In [7]:
contexts = bm25.retrieve(query='kant')
text_contexts = [context.text for context in contexts]

In [8]:
nl = ' \n ------ \n'
print(f"{nl}{nl.join(text_contexts)}")

Ces 10 paragraphes pertinents à propos d'Emmanuel Kant pourront être utilisés comme **"contextes"** dans la phase de modélisation. Mais avant de nous y intéresser, tentons de **visualiser les données** à notre dispositions. 

# Visualisation des données 📊

Afin d'observer certains graphiques intéressants concernant nos données, nous allons utiliser pour l'exemple les paragraphes pertinents à propos d'Emmanuel Kant extraits précédemment.

## Préparation préalable des données

Nous commençons par charger ces paragraphes dans un objet TextProcessing et par les "préparer" (la préparation consiste à retirer les espaces superflus en début et fin de paragraphe, à passer le texte en minuscule et à le diviser en *tokens*) afin que la visualisation soit plus aisée.

In [9]:
t_p = TextProcessing(retrieved_contexts = contexts)
t_p.load()
t_p.prepare()

Nous pouvons alors afficher les 10 premiers tokens ainsi obtenus par exemple.

In [10]:
t_p.text_split[:10]

## Fréquences de mots

Une fois les données chargées et préparées, il est possible de s'intéresser aux **fréquences des différents mots** (les 15 plus fréquents dans notre cas) dans l'ensemble des paragraphes pertinents à propos d'Emmanuel Kant. 

In [11]:
sorted_cardinalities = cardinality_of_words(t_p.text_split)
common_words = list(sorted_cardinalities.items())[:N_MOST_COMMON]
common_words

Nous pouvons **afficher sous forme de graphique** ces différentes fréquences.

In [12]:
words = [w[0] for w in common_words]
counts = [w[1] for w in common_words]
plt.style.use('dark_background')
plt.figure(figsize=(15, 9))
seaborn.barplot(x = words, y = counts).set_title("Mots les plus fréquents dans l'ensemble des paragraphes extraits");

Nous pouvons constater que des mots tels que *the*, *of*, *and* ou encore *in* figurent parmis les termes les plus fréquents. En recherche d'information, de tels mots sont appelés **mots vides** (ou *stop words*, en anglais). Il s'agit de mots tellement communs qu'il est inutile de les utiliser dans une recherche car ils sont peu instructifs sur le contexte étudié.

Nous pouvons donc afficher les fréquences de mots mais cette fois-ci en **supprimant les mots vides** afin de mettre en lumière les mots fréquents les plus pertinents et significatifs des paragraphes extraits.

In [13]:
text_without_stopwords = t_p.without_stopwords()
cardinalities = Counter(text_without_stopwords)
words = [cardinality[0] for cardinality in cardinalities.most_common(N_MOST_COMMON)]
counts = [cardinality[1] for cardinality in cardinalities.most_common(N_MOST_COMMON)]
plt.style.use('dark_background')
plt.figure(figsize=(15, 9))
seaborn.barplot(x = words, y = counts).set_title("Mots pleins les plus fréquents dans l'ensemble des paragraphes extraits");

Les mots ainsi affichés mettent en lumière la pertinence des paragraphes extraits par l'algorithme BM25 vis à vis du thème choisi à savoir Emmanuel Kant, *kant* étant largement le mot le plus fréquent. Les autres mots fréquents tels que *critique* et *reason* faisant echo à son oeuvre principale ou encore *philosophy* sont également représentatifs du thème choisi. 

Il est également possible de comparer la fréquence d'un même mot selon les différents paragraphes extraits sur un même thème. La cellule suivante permet d'afficher le nombre d'occurence du terme *kant* dans les 10 paragraphes extraits sous la forme d'un *waffle chart*.

In [14]:
fig = graph_occurrence("kant", contexts)

## Nuage de mots

Il est également possible de représenter les mots des paragraphes pertinents à propos d'Emmanuel Kant sous la forme de nuage de mots clés (ou *word clouds* en anglais). Le nuage de mots clés est une sorte de condensé sémantique d'un texte dans lequel les concepts clefs évoqués sont dotés d'une unité de taille (dans le sens du poids de la typographie utilisée) permettant de faire ressortir leur importance dans le texte.

In [16]:
wordcloud = WordCloud(width=800, height=500,
                      #random_state=21, max_font_size=110).generate(t_p.text)
plt.figure(figsize=(19, 12))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off');

Cette visualisation permet d'obtenir une sorte résumé des paragraphes pertinents dont on voit directement apparaître les idées clés et donc importantes.

## Word embedding (vectorisation des mots) et visualisation

Le **word embedding** est une méthode d'apprentissage d'une représentation de mots utilisée en traitement automatique des langues. Cette technique permet de représenter chaque mot d'un corpus de textes par un vecteur de nombres réels. Cette nouvelle représentation a ceci de particulier que **les mots apparaissant dans des contextes similaires possèdent des vecteurs correspondants qui sont relativement proches**. Par exemple, on pourrait s'attendre à ce que les mots « chien » et « chat » soient représentés par des vecteurs relativement peu distants dans l'espace vectoriel où sont définis ces vecteurs.

Nous pouvons donc utiliser un modèle de vectorisation de mots déjà entraîné puis des méthodes de réduction de dimmensions afin de visualiser dans l'espace latent des mots anglais, les mots fréquents des paragraphes extraits à propos d'Emmanuel Kant.

**Remarque** : Nous choisissons de ne vectoriser et représenter que quelques mots seulement dans un soucis de lisibilité du graphique.

In [17]:
e = Embedding(MODEL_NAME)
e.embedding(t_p.text[:301])
words_embedding_dim_2 = e.pca()

In [18]:
seaborn.set()
plt.style.use('dark_background')
plt.figure(figsize = (15, 9))
plt.plot(words_embedding_dim_2[:,0], words_embedding_dim_2[:,1], 'b', label = 'Vecteurs de mots', linewidth=0, marker = '*')
plt.title('Word Embedding sur les premiers mots des paragraphes extraits')
plt.xlabel('Première dimension')
plt.ylabel('Deuxième dimension')
plt.legend()
for i, w in enumerate(e.words):
    plt.annotate(w, xy=(words_embedding_dim_2[i, 0], words_embedding_dim_2[i, 1]))

plt.show()

Il apparaît par exemple que les termes *philosophers*, *kant* et *ideas* sont proches dans l'espace latent de mots ce qui est cohérent avec la sémantique. 

# Modélisation 🔍

À présent, nous pouvons nous intéresser à la véritable motivation de ce projet à savoir l'implémentation d'une *pipeline* permettant de **générer un quiz** (paires de questions/réponses) sur un thème donné. 

Les différentes composantes de la *pipeline* permettent de réaliser les étapes suivantes :

1. À partir des contextes (paragraphes) pertinents sur le thème donné (ici Emmanuel Kant) extraits de notre base textuelle initiale grâce au **BM25 Retriever**, nous pouvons extraire grâce à un outil de **Name Entity Recognition**, des noms propres, dates, lieux ou organisations susceptibles de consistuer des réponses potentielles à des questions.
2. Un générateur de questions appelé **T5** déjà pré-entraîné et *fine-tuné* nous permet de générer, en prenant en entrée un contexte et une réponse, une question cohérente.
3. La série de questions/réponses ainsi obtenue est ensuite filtrée pour ne conserver que les paires les plus pertinentes et correctes grâce à une méthode de filtrage appelée *Roundtrip* pensée et expliquée dans l'article suivant : https://aclanthology.org/P19-1620.pdf

## Extraction de réponses par reconnaissance d'entités nommées (Name Entity Recognition)

L'idée de cette première étape est d'extraire des réponses potentielles aux futures questions générées à partir des contextes pertinents sur le thème choisi (ici Kant en l'occurence).

Nous utilisons pour cela l'outil de reconnaissance d'entités nommées de l'Université de Standford appelé ```stanza``` que nous commençons par télécharger.

In [ ]:
project_dir = os.getcwd().split('src')[0]
sys.path.append(project_dir)
stanza.download('en', model_dir="data/stanza")

Puis à l'aide de la fonction implémentée ```extract_answers_from_contexts()```nous pouvons extraire des réponses potentielles dans les contextes pertinents sur Emmanuel Kant.

In [19]:
questions = [Question(retrieved_contexts = [context]) for context in contexts]
qca = QuestionContextAnswer(questions = questions)
qca = extract_answers_from_contexts(qca, "data/stanza")

En voici quelques exemples :

In [20]:
all_answers = [answer.text for answer in qca.get_all_answers()]
all_answers[:15]

## Génération de questions à partir des contextes et réponses

À partir des contextes et réponses extraites, nous pouvons générer à l'aide du modèle de langage **T5**, des questions cohérentes avec ces contextes/réponses. Pour ce faire nous utilisons un modèle pré-entraîné et *fine-tuné* disponible en open source (https://huggingface.co/Narrativa/mT5-base-finetuned-tydiQA-question-generation) utilisé dans notre fonction implémentée ```generate_questions()```.

In [21]:
qca = generate_questions(qca, 15, model_path="Narrativa/mT5-base-finetuned-tydiQA-question-generation")

Nous obtenons au final 67 paires de questions réponses différents à partir des différents contextes et réponses en entrée. 

In [22]:
print(f'Un exemple de paire de question/réponse pertinente : \n{qca.questions[15].text} / {qca.questions[15].predicted_answers[0].text}')

Au vue de la paire de question/réponse précédente, nous pourrions croire que l'ensemble des questions générées par le modèle **T5** sont aussi pertinentes. Néanmoins, ce modèle de langage comme tous les autres est loin d'être infaillible. Il peut générer des questions incohérentes avec la réponse extraite, en voici un exemple :

In [23]:
print(f'Un exemple de paire de question/réponse incorrecte : \n{qca.questions[1].text} / {qca.questions[1].predicted_answers[0].text}')

C'est pourquoi il est nécessaire de **filtrer les questions/réponses** ainsi obtenues afin de ne conserver que les plus correctes dans le quiz final.

## Filtrage des questions/réponses incorrectes

Le principe de cette étape de filtrage est de conserver uniquement les paires de questions/réponses les plus pertinentes dans le quiz final. Pour cela, nous appliquons un algorithme de filtrage dont les étapes sont les suivantes :
1. En prenant en entrée le contexte **C** et la question générée **Q**, un modèle de langage BERT fine-tuné pour la tâche de Question Answering (disponible en open-source au lien suivant https://huggingface.co/csarron/roberta-base-squad-v1) va "lire" la réponse à la question la plus vraissemblable au sein du contexte. Nous nommons cette nouvelle réponse **R'**.
2. Si la réponse **R** extraite par Name Entity Recognition et la réponse **R'** lue par le modèle BERT sont suffisamment proches (au sens de la distance Leveinshtein) alors nous considérons que la paire de question/réponse **Q/R** est pertinente. En effet, cela signifie vraissemblablement que la question **Q** est suffisamment cohérente pour que le modèle BERT ait pu lire une réponse similaire à celle initialement associée à cette question.

La cellule suivante permet d'appliquer ce filtre. 

In [24]:
qca = roundtrip_filter(qca, model_path="csarron/roberta-base-squad-v1")

In [25]:
print(f'Nouvelle paire de question réponse au rang 1 :\n{qca.questions[1].text} / {qca.questions[1].predicted_answers[0].text}')

Nous constatons que la paire de question/réponse incorrecte du rang 1 que nous avions précedemment affichée : 
> What was Kant's first philosophy? / Kant

a été remplacée par la paire suivante :
> Who is the most powerful philosopher? / Kant

Cela signifie que la première paire de question réponse a été filtrée ce qui laisse penser que le filtre est relativement performant pour éliminer les questions sémantiquement incorrectes vis à vis de leurs réponses.

# Génération d'un quiz 💡

Nous vous avons ainsi présenté les différentes étapes, algorithmes et modèles de langage utilisé dans ce projet. Nous pouvons à présent vous présenter **l'outil de génération de quiz** que nous avons élaboré et qui reprend l'ensemble des étapes que nous avons énoncé précédemment. Il permet de créer **un quiz de 10 questions/réponses** sur un thème choisi (ou moins si les circonstances ont fait que l'outil n'a pas pu générer 10 questions réponses (pauvreté des contextes, filtrage etc.) 

N'hésitez pas à changer le thème du quiz pour comparer les performances de notre outil et créer les quiz de votre choix.

In [26]:
quiz_generator(theme="kant")

# Pistes d'amélioration du projet

1. Notre objectif initial était de générer des paires de questions réponses en français. Pour cela nous aurions dû fonder notre modèle sur la version française de encyclopédie Wikipédia. Cependant, celle-ci étant trop volumineuse, elle n'était pas transferable sur le serveur GitHub ce qui rendait notre travail difficilement reproductible. C'est pourquoi nous nous sommes tournés vers la version *Simple English* moins riche linguistiquement mais suffisante pour utiliser les outils que nous avions implémentés. 

2. Nous souhaitions également *fine-tuner* nous-même les modèles de langages **T5** et **BERT** afin d'avoir une meilleure idée des performances finales de ces outils. Dans notre cas, nous avons utilisé ces modèles déjà *fine-tuné* par d'autres développeurs et disponibles en open-source. L'avantage a été un gain de temps considérable car l'entraînement d'un modèle de langage peut s'avérer très long mais en contre partie nous ne savons pas exactement comment ces modèles ont été entraînés et quelles sont leurs performances sur un dataset test : nous pouvons seulement évaluer leur robustesse à travers notre outil.